In [1]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
import base64
import numpy as np
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from keras.models import load_model

Using TensorFlow backend.


In [2]:
sio = socketio.Server()
app = Flask(__name__)

def send_control(steering_angle, throttle):
    sio.emit("steer", data={'steering_angle': str(steering_angle),
                            'throttle': str(throttle) }, skip_sid=True)

In [3]:
model = load_model('/home/witek/DataWorkshop/3rd_Edition/Day5/mymodel.h5')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalize (Lambda)           (None, 60, 80, 3)         0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 58, 78, 64)        1792      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 29, 39, 64)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 27, 37, 32)        18464     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 13, 18, 32)        0         
_______________________________________

In [5]:
def process_image(img):
    #160,320,3 ==> 60,80,3
    return img[10:130:2, ::4, :]

@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"].replace(',', '.'))
        image_str = data["image"]
        
        decoded = base64.b64decode(image_str)
        
        image = Image.open(BytesIO(decoded))
        image_array = np.asarray(image)
        print(image_array.shape)

        #plt.imshow(image_array)
        #plt.show()
        
        img = process_image(image_array)
        img_batch = np.expand_dims(img, axis=0)

        steering_angle = str(float(model.predict(img_batch))).replace('.', ',')

        throttle = '0,1'
        
        if speed < 15:
            throttle = '0,5'
        if speed > 17:
            throttle = '-0,1'

        send_control(steering_angle, throttle)
    else:
        sio.emit('manual', data={}, skip_sid=True)
        
app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(7215) wsgi starting up on http://0.0.0.0:4567
(7215) accepted ('127.0.0.1', 34153)


(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 320, 3)
(160, 

127.0.0.1 - - [17/May/2019 12:08:30] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 20.805822
wsgi exiting
(7215) wsgi exited, is_accepting=True
